In [ ]:
import csv
import pandas as pd
import matplotlib
import bar_chart_race as bcr
import helper_functions

In [ ]:
FILTER_YEAR_START = 1900
FILTER_YEAR_END = 2020

raw_data = []
unique_years = []
unique_teams = []

with open('data/results.csv') as results_file:
    reader = csv.reader(results_file)
    next(reader)
    for matchup in reader:
        # matchup[0] : date
        # matchup[1] : home team
        # matchup[2] : away team
        # matchup[3] : home score
        # matchup[4] : away score
        year = str(helper_functions.extract_date_to_year(matchup[0]))
        if int(year) < FILTER_YEAR_START or int(year) > FILTER_YEAR_END:
            continue
        raw_data.append(matchup)
        if year not in unique_years:
            unique_years.append(year)
        home_team = matchup[1]
        away_team = matchup[2]
        if home_team not in unique_teams:
            unique_teams.append(home_team)
        if away_team not in unique_teams:
            unique_teams.append(away_team)

unique_years = sorted(unique_years)
unique_teams = sorted(unique_teams) 

In [ ]:
# Data dictionary format:
# data_dict = {
#   '1940' : { 'England': 4 , 'Wales': 1, ... },
#   '1950' : { 'England': 6 , 'Wales': 3, ... },
#   ...
#   }
data_dict = {}

# Create an empty dict of dicts to be populated later
for y in unique_years: 
    empty_teams_list = {}
    for team in unique_teams:
        empty_teams_list[team] = 0
    data_dict[y] = empty_teams_list

# test dataframe
# df = pd.DataFrame(data_dict)
# df_t = df.T
# print(df_t)

In [ ]:
# Fill the dictionary by using the raw data
for match in raw_data:
    current_year = str(helper_functions.extract_date_to_year(match[0]))
    home_team = ''
    away_team = ''
    home_score = 0
    away_score = 0
    if (match[3]).upper() == 'NA' or (match[4]).upper() == 'NA' :
        continue
    home_team = match[1]
    away_team = match[2]
    home_score = int(match[3])
    away_score = int(match[4])
    # Home Team Win
    # print('{} : {} --- vs --- {} : {}'.format(home_team, home_score, away_team, away_score))
    if home_score > away_score:
        current_points = int(data_dict[current_year][home_team])
        # print('For {} - adding to {}'.format(current_year, home_team))
        current_points += 3
        data_dict[current_year][home_team] = current_points        
    # Away Team Win
    elif home_score < away_score:
        current_points = int(data_dict[current_year][away_team])
        # print('For {} - adding to {}'.format(current_year, away_team))
        current_points += 3
        data_dict[current_year][away_team] = current_points        
    # Draw
    elif home_score == away_score:
        # noOp
        current_points_home = int(data_dict[current_year][home_team])
        current_points_home += 1
        current_points_away = int(data_dict[current_year][away_team])
        current_points_away += 1
        data_dict[current_year][home_team] = current_points_home        
        data_dict[current_year][away_team] = current_points_away        

        
    else:
        print('ERROR: Match score unreadable')

# print(data_dict)
# test dataframe
# df = pd.DataFrame(data_dict)
# df_t = df.T
# print(df_t)

In [ ]:
points_temp = empty_teams_list
# print(points_temp)
for year in data_dict:
    for team in data_dict[year]:
        data_dict[year][team] += int(points_temp[team])
    points_temp = data_dict[year]

# print(data_dict)

data_frame_inital = pd.DataFrame(data_dict)
data_frame_t = data_frame_inital.T
# print(df_t)
    

In [ ]:
# bcr.bar_chart_race(
#     df=data_frame_t,
#     filename='teams_wins_race.mp4',
#     orientation='h',
#     sort='desc',
#     n_bars=7,
#     fixed_order=False,
#     fixed_max=True,
#     steps_per_period=10,
#     interpolate_period=False,
#     label_bars=True,
#     bar_size=.95,
#     period_label={'x': .99, 'y': .25, 'ha': 'right', 'va': 'center'},
#     period_fmt='',
#     period_summary_func=lambda v, r: {'x': .99, 'y': .18,
#                                     #   's': f'Total wins: {v.nlargest(6).sum():,.0f}',
#                                       's': '',
#                                       'ha': 'right', 'size': 8, 'family': 'Helvetica'},
#     perpendicular_bar_func='median',
#     period_length=500,
#     figsize=(5, 3),
#     dpi=256,
#     cmap='dark12',
#     title='Wins by international teams',
#     title_size='',
#     bar_label_size=7,
#     tick_label_size=7,
#     shared_fontdict={'family' : 'Helvetica', 'color' : '.1'},
#     scale='linear',
#     writer=None,
#     fig=None,
#     bar_kwargs={'alpha': .7},
#     filter_column_colors=True)  

In [ ]:
bcr.bar_chart_race(
    df=data_frame_t, 
    filename='teams_points_race.mp4',
    sort='desc', 
    steps_per_period=20, 
    figsize=(7, 4), 
    n_bars=7,
    title='Points accumulated by international teams',
    filter_column_colors=True,
    dpi=512,
    scale='linear',
    bar_size=.85,
    # period_label={'x': .99, 'y': .25, 'ha': 'right', 'va': 'center'},
    bar_label_size=7,
    tick_label_size=7,
    )